In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
def create_random_windows(group, window_size):
    max_possible = group.shape[0] - window_size
    if max_possible < 1:
        return None  # Not enough data to create a window
    start_index = np.random.randint(0, max_possible)
    return group.iloc[start_index:start_index + window_size]

# Metadata for the windowing and network
num_device_categories = 3
num_input_parameters = 47
num_recur_hours = 24

# Group by house number and apply the function
random_windows = data_sorted.groupby('HouseNum').apply(create_random_windows)
random_windows = random_windows.dropna()  # Remove groups that didn't have enough data

In [ ]:
# Two layer LSTM model
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(num_recur_hours, num_input_parameters)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(num_device_categories, activation='softmax')
])

# Adam optimizer, crossentropy loss
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()